In [ ]:
# Codes are adapted from http://bit.ly/2U7ttYT (파이토치 첫걸음,한빛미디어)

# VGG19 Implementation

- VGG-19
- Convolution layer
- Maxpooling layer
- Fully connected layer


## 1. Settings
### 1) Import required libraries

In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### Hyperparameter

In [54]:
batch_size= 100
learning_rate = 0.001
num_epoch = 50

### 2. Data Loader


                    

In [40]:

# 이때 이미지들에 대한 변형도 같이 전달해줍니다.
transform_train = transforms.Compose([
            transforms.Resize(32),                   # 이미지 크기를 32x32으로 바꿔줍니다.
            transforms.RandomCrop(32, padding=4),        # 32x32 이미지의 랜덤한 위치에서 30x30 크기만큼 샘플링 합니다.
            transforms.RandomHorizontalFlip(),        # 랜덤한 확률로 이미지를 좌우반전 합니다.
            transforms.ToTensor(),                # 이미지 데이터를 텐서로 변형합니다.
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
             ])

transform_test = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])


In [41]:
# CIFAR10 dataset
train_dataset = datasets.CIFAR10(root='./data/',
                                           train=True, 
                                           transform=transform_train,
                                           download=True)

test_dataset = datasets.CIFAR10(root='./data/',
                                          train=False, 
                                          transform=transform_test)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

Files already downloaded and verified


## 3. Model 
### 1) Basic Blocks

- 모델에 반복되는 부분이 많기 때문에 이를 함수로 만들어 단순화 합니다.
- 맨 위에 이미지를 보면 컨볼루션 연산이 2번 연속하는 경우와 3번 연속하는 경우가 있는데 이를 각각 만들어줍니다.
- 아래의 코드는 최적의 방법이라기 보다는 그림의 구조를 모방한 코드입니다. 

In [42]:
# 컨볼루션 연산이 2번 연속하는 경우
# 컨볼루션-활성화함수-컨볼루션-활성화함수-풀링
def conv_2_block(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1), # 인푹풋채널,아웃푹채널
        nn.BatchNorm2d(out_dim),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.BatchNorm2d(out_dim),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

  
# 컨볼루션 연산이 3번 연속하는 경우
# 컨볼루션-활성화함수-컨볼루션-활성화함수-컨볼루션-활성화함수-풀링

def conv_4_block(in_dim,out_dim): ## 노드안에 몇개있는지에따라 구현함
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,kernel_size=3,padding=1),
        nn.BatchNorm2d(out_dim),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.BatchNorm2d(out_dim),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.BatchNorm2d(out_dim),
        nn.ReLU(),
        nn.Conv2d(out_dim,out_dim,kernel_size=3,padding=1),
        nn.BatchNorm2d(out_dim),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )
    return model

### 2) VGG Model

In [43]:
# 위에서 정의한 블록들을 이용해 VGG 네트워크를 만들어보겠습니다.
# 필터의 개수가 2의 n승의 값을 가지기 때문에 base_dim이란 변수를 추가해서 단순화 했습니다.
# 현재 dog, cat 두 가지 클래스를 구분하려고 하기 때문에 num_classes=2로 설정했습니다.

class VGG(nn.Module):
    def __init__(self, base_dim, num_classes=10):
        super(VGG, self).__init__()
        self.feature = nn.Sequential(
            conv_2_block(3,base_dim), # 32 -> 16(maxpooling)
            conv_2_block(base_dim,2*base_dim), # 16 -> 8
            conv_4_block(2*base_dim,4*base_dim), # 8 -> 4
            conv_4_block(4*base_dim,8*base_dim), # 4 -> 2
            conv_4_block(8*base_dim,8*base_dim), # 2 -> 1      
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(8*base_dim * 1 * 1, 1000), # 8*base_dim * 1 * 1 : 입력  1000 : 출력은 지맘대로
            nn.ReLU(True),                                   # True 는 inplace 연산을 하겠다는 의미를 가집니다. inplace 연산은 결과값을 새로운 변수에 값을 저장하는 대신 기존의 데이터를 대체하는것을 의미합니다.
            nn.Dropout(p=0.2),  
            nn.Linear(1000, 500),
            nn.ReLU(True),
            nn.Dropout(p=0.2),
            nn.Linear(500, num_classes), # 분류해야하는 개수
        )

    def forward(self, x):
        x = self.feature(x) 
        x = x.view(x.size(0), -1)                                               # x.size(0)를 batch size로 바꿔도 같은 값입니다.
        x = self.fc_layer(x)
        return x
    

In [44]:
32*32

1024

## 4. Optimizer & Loss

In [50]:
# gpu가 사용 가능한 경우에는 device를 0번 gpu로 설정하고 불가능하면 cpu로 설정합니다.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# 앞서 정의한대로 vGG 클래스를 인스턴스화 하고 지정한 장치에 올립니다.
model = VGG(base_dim=64) 
model = model.to(device) 

# 손실함수 및 최적화함수를 설정합니다.
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 모델 자녀 노드의 이름과 모듈을 출력합니다.
for i in model.named_children():
    print(i)

cuda:0
('feature', Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Con

## 5. Train

In [56]:
for i in range(num_epoch):
    correct = 0
    total = 0
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)
        optimizer.zero_grad()
        outputs = model.forward(x)
        loss = loss_func(outputs,y_)
        loss.backward()
        optimizer.step()
        total += y_.size(0)
        _, predicted = torch.max(outputs.data, 1) # _는 받긴받는데 안쓰겠다 그리고 결과 확률들중에 가장높은확률의 클래스 확률과 레이블을 반환하겠다 이코드에서 확률은 변환안함
        correct += (predicted == y_).sum().item() # print 형태로 출력해보기위해 쓰는 코드
        if i%10 == 0:
            print('loss is ',loss)
            print('Test Accuracy is : {} %'.format(100 * correct / total))
       

loss is  tensor(0.0641, device='cuda:0', grad_fn=<NllLossBackward>)
Test Accuracy is : 99.0 %
loss is  tensor(0.0710, device='cuda:0', grad_fn=<NllLossBackward>)
Test Accuracy is : 98.5 %
loss is  tensor(0.1092, device='cuda:0', grad_fn=<NllLossBackward>)
Test Accuracy is : 98.0 %
loss is  tensor(0.0678, device='cuda:0', grad_fn=<NllLossBackward>)
Test Accuracy is : 98.0 %
loss is  tensor(0.0396, device='cuda:0', grad_fn=<NllLossBackward>)
Test Accuracy is : 98.2 %
loss is  tensor(0.2067, device='cuda:0', grad_fn=<NllLossBackward>)
Test Accuracy is : 97.5 %
loss is  tensor(0.0739, device='cuda:0', grad_fn=<NllLossBackward>)
Test Accuracy is : 97.42857142857143 %
loss is  tensor(0.0855, device='cuda:0', grad_fn=<NllLossBackward>)
Test Accuracy is : 97.5 %
loss is  tensor(0.0192, device='cuda:0', grad_fn=<NllLossBackward>)
Test Accuracy is : 97.77777777777777 %
loss is  tensor(0.1460, device='cuda:0', grad_fn=<NllLossBackward>)
Test Accuracy is : 97.6 %
loss is  tensor(0.1469, device='cu

## 모델 저장하기

In [32]:
PATH = './saveModel/'
torch.save(model.state_dict(), PATH+'VGG19_CIFAR10_state')

In [33]:
# 불러오기
model1 = model.load_state_dict(torch.load(PATH+'VGG19_CIFAR10_state'))

### 전체 모델 저장하기/불러오기

In [36]:
torch.save(model, PATH+'VGG19_CIFAR10_orgin')

In [37]:
model1 = torch.load(PATH+'VGG19_CIFAR10_orgin')

## test

In [57]:
# Test the model again using the saved weight file
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 89.48 %
